In [1]:
import torch
import torchsummary

In [2]:
import numpy as np

In [3]:
from torch import nn

In [4]:
layer1 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)

In [5]:
batch_norm1 = nn.BatchNorm2d(num_features=64)

In [6]:
prelu1 = nn.PReLU()

In [7]:
layer2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)

In [8]:
batch_norm2 = nn.BatchNorm2d(num_features=64)

In [9]:
class ResBlock(nn.Module):
    def __init__(self,kernel_size,channels,padding,stride):
        super(ResBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.batch_norm1 = nn.BatchNorm2d(num_features=channels)
        self.prelu1 = nn.PReLU()
        self.conv2 = nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.batch_norm2 = nn.BatchNorm2d(num_features=channels)
    def forward(self,inp):
        x = self.conv1(inp)
        x = self.batch_norm1(x)
        x = self.prelu1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        output = x + inp
        return output

In [10]:
resblock = ResBlock(3,64,1,1)

In [11]:
inp = torch.randn(1,64,2,2)

In [12]:
resblock(inp).shape

torch.Size([1, 64, 2, 2])

In [13]:
class SubPixelConv(nn.Module):
    def __init__(self,kernel_size,in_channels,out_channels,padding,stride):
        super(SubPixelConv,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.pixel_shuffler1 = nn.PixelShuffle(2)
        self.prelu1 = nn.PReLU()
    def forward(self,inp):
        x = self.conv1(inp)
        x = self.pixel_shuffler1(x)
        output = self.prelu1(x)
        return output

In [16]:
subpixelconv = SubPixelConv(3,64,256,1,1)

In [17]:
subpixelconv(resblock(inp)).shape

torch.Size([1, 64, 4, 4])

In [18]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,kernel_size=9,out_channels=64,padding=4,stride=1)
        self.prelu1 = nn.PReLU()
        self.resblock1 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock2 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock3 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock4 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock5 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock6 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock7 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock8 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock9 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock10 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock11 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock12 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock13 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock14 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock15 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.resblock16 = ResBlock(channels=64,kernel_size=3,padding=1,stride=1)
        self.conv2 = nn.Conv2d(in_channels=64,kernel_size=3,out_channels=64,padding=1,stride=1)
        self.batch_norm1 = nn.BatchNorm2d(num_features=64)
        self.subpixelconv1 = SubPixelConv(in_channels=64,kernel_size=3,out_channels=256,padding=1,stride=1)
        self.subpixelconv2 = SubPixelConv(in_channels=64,kernel_size=3,out_channels=256,padding=1,stride=1)
        self.conv3 = nn.Conv2d(in_channels=64,kernel_size=9,out_channels=3,padding=4,stride=1)
    
    def forward(self,inp):
        x = self.conv1(inp)
        x = self.prelu1(x)
        y = self.resblock1(x)
        y = self.resblock2(y)
        y = self.resblock3(y)
        y = self.resblock4(y)
        y = self.resblock5(y)
        y = self.resblock6(y)
        y = self.resblock7(y)
        y = self.resblock8(y)
        y = self.resblock9(y)
        y = self.resblock10(y)
        y = self.resblock11(y)
        y = self.resblock12(y)
        y = self.resblock13(y)
        y = self.resblock14(y)
        y = self.resblock15(y)
        y = self.resblock16(y)
        y = self.conv2(y)
        y = self.batch_norm1(y)
        y = self.subpixelconv1(x+y)
        y = self.subpixelconv2(y)
        output = self.conv3(y)
        return output

In [19]:
generator = Generator()

In [23]:
model = nn.Sequential()
model.add_module(module=generator,name='generator')

In [ ]:
torchsummary.summary(model,input_size=(3,56,56))

In [35]:
model.forward(inp)

tensor([[[[ 1.3964, -0.2138],
          [ 0.7101, -1.4748]],

         [[-0.6391,  1.4618],
          [ 1.2519,  0.0663]],

         [[-0.3244, -0.7537],
          [-1.5978, -0.4525]],

         [[ 0.4607,  0.1563],
          [ 1.0554, -2.6950]],

         [[-0.9548, -0.1770],
          [ 0.2713, -0.7973]],

         [[ 2.3619,  0.3170],
          [-0.9167, -0.2072]],

         [[ 0.0494, -2.0885],
          [ 0.3592, -0.2938]],

         [[ 0.5089,  0.4162],
          [ 0.2064, -1.8366]],

         [[-0.4067, -0.0246],
          [-1.3946, -0.6146]],

         [[ 0.3186, -1.3618],
          [ 2.6360,  0.5912]],

         [[-2.0839, -0.0138],
          [ 2.6350,  0.1775]],

         [[ 1.3910, -0.7048],
          [ 0.1877, -1.1083]],

         [[ 0.6507,  0.3739],
          [ 0.0545,  0.2233]],

         [[-0.1060,  2.7515],
          [ 0.7918, -0.2241]],

         [[-0.3089, -1.1391],
          [-0.0386,  0.6891]],

         [[-1.0344, -1.2680],
          [-0.0309, -1.0123]],

        

In [44]:
resblock = list(model.children())[0]

In [47]:
resblock.forward(inp) - model.forward(inp)

tensor([[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 

In [51]:
layers = list(resblock.children())

In [52]:
x = inp
for layer in layers:
    x = layer(x)

In [54]:
x+inp - resblock.forward(inp)

tensor([[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 